In [1]:
try:
  df_platform
  print("Data already loaded")
except NameError:
  print("Loading data...")
  %run data.ipynb

Loading data...


/var/folders/qf/j616b9d53yn77zcpky42s5r40000gn/T/ipykernel_95999/933785469.py:6: DtypeWarning: Columns (20,28,29,30,31,32,33,40,41,42,43,45,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stripe_payouts = pd.read_csv(f'data/{host}/{year}/stripe_payouts.csv')


In [2]:
df_bank_reconciliation = df_banks.copy()
df_bank_reconciliation.head()

,index,Post Date,Customer Ref,Debit,Credit,BAI Description,Detail,Account Number
0,0,2023-01-03,Google LLC ED,NaN,10000.00,PREAUTHORIZED ACH CREDIT,Google LLC EDI PYMNTS 20230103 ...,4869374308
1,1,2023-01-03,007000103062636,NaN,90.00,INCOMING MONEY TRANSFER,INCOMING FLS ACCT#4869374308 1/MICHAEL ...,4869374308
2,2,2023-01-03,007000103103130,NaN,5.00,INCOMING MONEY TRANSFER,INCOMING FLS ACCT#4869374308 1/DAVIDE F...,4869374308
3,3,2023-01-03,VELOCITY GLOBAL ED,3401.28,NaN,PREAUTHORIZED ACH DEBIT,VELOCITY GLOBAL EDI PAYMNT 20230102 ...,4869374308
4,4,2023-01-04,Wise Inc 61,NaN,159.61,PREAUTHORIZED ACH CREDIT,Wise Inc 610275 610275 ...,4869374308


In [3]:
df_platform_bank_related = df_platform[(df_platform['kind'] == 'CONTRIBUTION') | (df_platform['kind'] == 'EXPENSE') | (df_platform['kind'] == 'ADDED_FUNDS')]
print(f"{len(df_platform_bank_related)} bank related transactions out of {len(df_platform)} total transactions ({len(df_platform_bank_related)/len(df_platform)*100:.2f}%)")

204025 bank related transactions out of 845230 total transactions (24.14%)


In [4]:
import pandas as pd
from utils import matchByDateAndValue
df_platform_bank_debits = matchByDateAndValue(df_platform_bank_related, ['createdAt', 'amountInHostCurrency.value'], df_bank_reconciliation, ['Post Date', 'Debit'] , '2 days', 0.05, True).dropna(subset=['df2key'])
df_platform_bank_credits = matchByDateAndValue(df_platform_bank_related, ['createdAt', 'amountInHostCurrency.value'], df_bank_reconciliation, ['Post Date', 'Credit'] , '2 days', 0.05, True).dropna(subset=['df2key'])
df_platform_bank = pd.concat([df_platform_bank_debits, df_platform_bank_credits])
print('Successfully merged {} rows out of {} from Value and Date'.format(len(df_platform_bank), len(df_bank_reconciliation)))

Successfully merged 400 rows out of 660 from Value and Date


In [5]:
df_left_bank = df_bank_reconciliation[~df_bank_reconciliation['index'].isin(df_platform_bank['index'])]
df_payouts_bank = matchByDateAndValue(df_stripe_payouts, ['automatic_payout_effective_at_utc', 'net'], df_left_bank, ['Post Date', 'Credit'] , '2 days', 0.05, True).dropna(subset=['df2key'])
df_left_bank = df_left_bank[~df_left_bank['index'].isin(df_payouts_bank['index'])]
print('Successfully merged {} rows out of {} from payouts Value and Date'.format(len(df_payouts_bank), len(df_left_bank)))

Successfully merged 10 rows out of 243 from payouts Value and Date
